## [Observability for OpenAI Assistants API with Langfuse ](https://langfuse.com/guides/cookbook/integration_openai_assistants)


how to use the Langfuse observe decorator to trace calls made to the [OpenAI Assistants API](https://platform.openai.com/docs/assistants/migration). It covers creating an assistant, running it on a thread, and observing the execution with Langfuse tracing.

Note: The native OpenAI SDK wrapper does not support tracing of the OpenAI assistants API, you need to instrument it via the decorator as shown in this notebook.

### Step 0: Setup

In [ ]:
%pip install --upgrade openai langfuse

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Attempting uninstall: langfuse
    Found existing installation: langfuse 2.60.9
    Uninstalling langfuse-2.60.9:
      Successfully uninstalled langfuse-2.60.9
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langfuse-haystack 2.2.0 requires langfuse<3.0.0,>=2.9.0, but you have langfuse 3.3.4 which is incompatible.


In [1]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-962190cc-b2bd-47c0-b752-8de287a2a5c1"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-4e9959c3-0935-4142-b789-734beb81d15a"
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

# Your openai key
# os.environ["OPENAI_API_KEY"] = "f0c1fb9f5c534e55a66d9e539916fdb0.GQKa6HaX6MpT9ioJ"
os.environ["OPENAI_API_KEY"] = "73c80b33ad68446ea3f059efe5c1a65f.T2PZjYiHcT2JYx2a"
os.environ["OPENAI_BASE_URL"] = "https://open.bigmodel.cn/api/paas/v4"

### Step1: Create an Assistant

Use the client.beta.assistants.create method to create a new assistant. 

In [ ]:
# --Fail
from langfuse import observe
from openai import OpenAI

@observe()
def create_assistant():
    client = OpenAI()
    assistant = client.beta.assistants.create(
        name="Math Tutor",
        instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
        model="glm-4.5-flash"
    )
    return assistant

assistant = create_assistant()
print(f"Created assistant: {assistant.id}")

NotFoundError: Error code: 404 - {'timestamp': '2025-09-12T06:05:13.751+00:00', 'status': 404, 'error': 'Not Found', 'path': '/v4/assistants'}

### Step 2: Running the Assistant

Create a thread and run the assistant on it:

In [3]:
@observe()
def run_assistant(assistant_id, user_input):
    client = OpenAI()

    thread = client.beta.threads.create()

    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="assistant",
        content="I am a math tutor that likes to help math students, how can I help?",
    )

    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_input
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

    return run, thread

user_input = "I need to solve the equation `3x + 11 = 14`. Can you help me?"
run, thread = run_assistant(assistant.id, user_input)
print(f"Created run: {run.id}")

NameError: name 'assistant' is not defined

### Step 2: Getting the Response

In [ ]:
import json
from langfuse import get_client
langfuse = get_client()

@observe()
def get_response(thread_id, run_id):
    client = OpenAI()

    messages = client.beta.threads.messages.list(thread_id=thread_id, order="asc")
    assistant_response = messages.data[-1].content[0].text.value
    # get run for token counts
    run_log = client.beta.threads.runs.retrieve(
        thread_id=thread_id,
        run_id=run_id
    )
 
    message_log = client.beta.threads.messages.list(
        thread_id=thread_id,
    )

    input_messages = [{"role": message.role, "content": message.content[0].text.value} for message in message_log.data[::-1][:-1]]

    langfuse_client = langfuse.client_instance
    # pass trace_id and current observation ids to the newly created child generation
    langfuse_client.generation(
        trace_id=langfuse.get_current_trace_id(),
        parent_observation_id=langfuse.get_current_observation_id(),
        model=run.model,
        usage_details=run.usage,
        input=input_messages,
        output=assistant_response
    )

    return assistant_response, run

response = get_response(thread.id, run.id)
print(f"Assistant response: {response[0]}")

: 

### All in one trace

In [ ]:
import time
 
@observe()
def run_math_tutor(user_input):
    assistant = create_assistant()
    run, thread = run_assistant(assistant.id, user_input)
 
    time.sleep(5) # notebook only, wait for the assistant to finish
 
    response = get_response(thread.id, run.id)
    
    return response[0]
 
user_input = "I need to solve the equation `3x + 11 = 14`. Can you help me?"
response = run_math_tutor(user_input)
print(f"Assistant response: {response}")